In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/examples/tuning/getting_started_tuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/tuning/getting_started_tuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/tuning/getting_started_tuning.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


## Tuning and deploy a foundation model

Creating an LLM requires massive amounts of data, significant computing resources, and specialized skills. On Vertex AI, tuning allows you to customize a foundation model for more specific tasks or knowledge domains.

While the prompt design is excellent for quick experimentation, if training data is available, you can achieve higher quality by tuning the model. Tuning a model enables you to customize the model response based on examples of the task you want the model to perform.

For more details on tuning have a look at the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

### Objective

This tutorial teaches you how to tune a foundational model on new unseen data and you will use the following Google Cloud products:

- Vertex AI Generative AI Studio
- Vertex AI Pipelines
- Vertex AI Model Registry
- Vertex AI Endpoints

The steps performed include:

- Get training data from BQ and generate a JSONL file
- Upload training data
- Create a pipeline job
- Inspect your model on Vertex AI Model Registry
- Get predictions from your tuned model

### Quota
**important**: Tuning the text-bison@001  model uses the tpu-v3-8 training resources and the accompanying quotas from your Google Cloud project. Each project has a default quota of eight v3-8 cores, which allows for one to two concurrent tuning jobs. If you want to run more concurrent jobs you need to request additional quota via the [Quotas page](https://console.cloud.google.com/iam-admin/quotas).

### Costs
This tutorial uses billable components of Google Cloud:

* Vertex AI Generative AI Studio

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Install Vertex AI SDK

In [ ]:
!pip install tensorboard

In [ ]:
!pip install google-cloud-aiplatform google-cloud-bigquery sequence-evaluate sentence-transformers rouge --upgrade --user

### BigQuery IAM
Now you need to add permissions to the service account:
- Go to the [IAM page](https://console.cloud.google.com/iam-admin/) in the console
- Look for the default compute service account. It should look something like this: `<project-number>-compute@developer.gserviceaccount.com`
- Assign the default compute service account with `bigquery.user`

### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`. Otherwise, check the support page: Locate the [project ID](https://support.google.com/googleapi/answer/7014113). Please update `PROJECT_ID` below.

In [2]:
PROJECT_ID = "pl-del-gen-ai"

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


### Create a bucket
Now you have to create a bucket that we will use to store our tuning data. To avoid name collisions between users on resources created, you generate a UUID for each instance session and append it to the name of the resources you create in this tutorial.

In [3]:
import random
import string

# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))

UUID = generate_uuid()

Choose a bucket name and update the `BUCKET_NAME` parameter.

In [4]:
BUCKET_NAME = "genai-workshop-123"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
REGION = "us-central1"  # @param {type: "string"}

In [5]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = "vertex-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

Only if your bucket doesn't already exist: Run the following cell to create your Cloud Storage bucket.

In [8]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://genai-workshop-123/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [6]:
! gsutil ls -al $BUCKET_URI

  24130380  2023-08-04T07:45:49Z  gs://genai-workshop-123/tune_data_stack_overflow_python_qa.jsonl#1691135149094394  metageneration=1
TOTAL: 1 objects, 24130380 bytes (23.01 MiB)


### Import libraries

In [7]:
from typing import Union

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

from vertexai.preview.language_models import TextGenerationModel
from google.cloud import aiplatform
from google.cloud import bigquery

from IPython.display import display, HTML, Markdown

## Tune your Model

Now it's time for you to create a tuning job. Tune a foundation model by creating a pipeline job using Generative AI Studio, cURL, or the Python SDK. In this notebook, we will be using the Python SDK. You will be using a Q&A with a context dataset in JSON format.

### Training Data
💾 Your model tuning dataset must be in a JSONL format where each line contains a single training example. You must make sure that you include instructions.

You will use the StackOverflow data on BigQuery Public Datasets, limiting to questions with the `python` tag, and accepted answers for answers since 2020-01-01.

First create a helper function to let you easily query BigQuery and return the results as a Pandas DataFrame.

In [8]:
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    
    return df

Next define the query.

In [9]:
df = run_bq_query("""SELECT
    CONCAT(q.title, q.body) as input_text,
    a.body AS output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    10000
""")

df.head()

Finished job_id: 51abc517-cf2d-4320-bc96-f0b37b03ef3e


,input_text,output_text
0,How to import (install) dependencies (packages...,<p>This is useful in a case that anyhow you ha...
1,High accuracy on LSTM-RNN model implemented on...,<p>I remember this article from years ago (it'...
2,'import' is not recognized as an internal or e...,<p>You're running <code>import cx_Oracle</code...
3,"ValueError: Input 0 of layer ""sequential_1"" is...",<p>Here is my attempt at a solution. I am not ...
4,"""IndentationError: expected an indented block""...",<p>Replace your comments in line 23 - 30 with ...


There should be 10k questions and answers.

In [10]:
print(len(df))

10000


Lets split the data into training and evalation. For Extractive Q&A tasks we advise 100+ training examples. In this case you will use 800.

In [11]:
display(HTML(df["input_text"][0]))

In [12]:
display(HTML(df["output_text"][0]))

In [13]:
# split is set to 80/20
train, evaluation = train_test_split(df, test_size=0.2)
len(train), len(evaluation)

(8000, 2000)

For tuning, the training data first needs to be converted into a JSONL format.

In [14]:
tune_jsonl = train.to_json(orient='records', lines=True)

with open("data.jsonl", "a") as f:
    f.write(tune_jsonl[0:10_000])

Next, you can write it to a local JSONL before transferring it to Google Cloud Storage (GCS).

In [15]:
training_data_filename = "tune_data_stack_overflow_python_qa.jsonl"

with open(training_data_filename, "w") as f:
    f.write(tune_jsonl)

You can then export the local file to GCS, so that it can be used by Vertex AI for the tuning job.

In [39]:
! gsutil cp $training_data_filename $BUCKET_URI

Copying file://tune_data_stack_overflow_python_qa.jsonl [Content-Type=application/octet-stream]...
/ [1 files][ 23.0 MiB/ 23.0 MiB]                                                
Operation completed over 1 objects/23.0 MiB.                                     


You can check to make sure that the file successfully transferred to your Google Cloud Storage bucket:

In [16]:
! gsutil ls -al $BUCKET_URI

  24130380  2023-08-04T07:45:49Z  gs://genai-workshop-123/tune_data_stack_overflow_python_qa.jsonl#1691135149094394  metageneration=1
TOTAL: 1 objects, 24130380 bytes (23.01 MiB)


In [17]:
TRAINING_DATA_URI = f"{BUCKET_URI}/{training_data_filename}"

### Model Tuning
Now it's time to start to tune a model. You will use the Vertex AI SDK to submit our tuning job.

#### Recommended Tuning Configurations
✅ Here are some recommended configurations for tuning a foundation model based on the task, in this example Q&A. You can find more in the [documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

Extractive QA:
- Make sure that your train dataset size is 100+
- Training steps [100-500]. You can try more than one value to get the best performance on a particular dataset (e.g. 100, 200, 500)

In [18]:
MODEL_NAME= f"genai-workshop-tuned-model-{UUID}"

In [19]:
import vertexai
from google.auth import default
credentials, _ = default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
#Function that starts the tuning job
def tuned_model(
    project_id: str,
    location: str,
    training_data: str,
    model_display_name: str,
    train_steps = 100,
    learning_rate = 3e-4,
):

    """Prompt-tune a new model, based on a prompt-response data.

    "training_data" can be either the GCS URI of a file formatted in JSONL format
    (for example: training_data=f'gs://{bucket}/{filename}.jsonl'), or a pandas
    DataFrame. Each training example should be JSONL record with two keys, for
    example:
      {
        "input_text": <input prompt>,
        "output_text": <associated output>
      },

    Args:
      project_id: GCP Project ID, used to initialize aiplatform
      location: GCP Region, used to initialize aiplatform
      training_data: GCS URI of training file or pandas dataframe of training data
      model_display_name: Name for your model. 
      train_steps: Number of training steps to use when tuning the model
    """

    vertexai.init(project=project_id, location=location, credentials=credentials)
    model = TextGenerationModel.from_pretrained("text-bison@001")
    model.tune_model(
        learning_rate=learning_rate,
        training_data=training_data,
        train_steps=train_steps,
        tuning_job_location="europe-west4",
        tuned_model_location="us-central1",
        model_display_name=model_display_name
    )

    print(model._job.status)
    return model

Next it's time to start your tuning job. **Disclaimer:** tuning and deploying a model takes time.

#### Start tuning

In [44]:
import math
n_epochs = 1
batch_size = 24
n_steps = math.floor(n_epochs * len(train) / batch_size)
n_steps

333

In [ ]:
# This will start the tuning job and output a URL where you can monitor the pipeline execution.
model = tuned_model(PROJECT_ID, REGION, TRAINING_DATA_URI, MODEL_NAME, n_steps)

Following the link above, you can view your pipeline run. As you can see in the screenshot below, it will execute the following steps:

- Validation
- Export managed dataset
- Convert JSONL to TFRecord
- Large language model tuning
- Upload LLM Model

## View your tuned foundational model on Vertex AI Model registry
When your tuning job is finished, your model will be available on Vertex AI Model Registry. The following Python SDK sample shows you how to list tuned models.

In [20]:
def list_tuned_models(project_id, location):
    vertexai.init(project=project_id, location=location, credentials=credentials)
    model = TextGenerationModel.from_pretrained("text-bison@001")
    tuned_model_names = model.list_tuned_model_names()
    print(tuned_model_names)

In [21]:
list_tuned_models(PROJECT_ID, REGION)

[]


You can also use the Google Cloud Console UI to view all of your model in [Vertex AI Model Registry](https://console.cloud.google.com/vertex-ai/models?e=13802955&jsmode=O&mods=-ai_platform_fake_service&project=cloud-llm-preview1). Below you can see an example of a tuned foundational model available on Vertex AI Model Registry.

## Use your tuned model to get predictions
Now it's time to get predictions. First you need to get the latest tuned model from the Vertex AI Model registry.

In [22]:
ID = "3322805502999527424"
model_path = f"projects/{PROJECT_ID}/locations/{REGION}/models/{ID}"

In [23]:
deployed_model = TextGenerationModel.get_tuned_model(model_path)
model = TextGenerationModel.from_pretrained("text-bison@001")

Now you can start send a prompt to the API. Feel free to update the following prompt.

In [79]:
q = evaluation["input_text"].reset_index(drop=True)[101]
display(Markdown(q))

why output of 'for in one line' is printed in above lines?<p>my code is....</p>
<pre><code>names = ['ali', 'parham', 'hasan', 'farhad']
print('names: ', [print(i) for i in names])
</code></pre>
<p>and my output is this...</p>
<pre><code>ali
parham
hasan
farhad
names:  [None, None, None, None]
</code></pre>
<p>but i want this....</p>
<pre><code>names: ali, parham, hasan, farhad
</code></pre>
<p>why what are in the list is 'None'?!</p>
<p>and why the name are in the above lines?!</p>

In [80]:
display(Markdown(deployed_model.predict(q, temperature=0.2).text))

<p>The output of the <code>print</code> function is <code>None</code>. So, the list contains <code>None</code>. You can use <code>list(map(print, names))</code> to get the output you want.</p>

In [81]:
display(Markdown(model.predict(q, max_output_tokens=1024, temperature=0.2).text))

The output of the `for` loop is printed in the above lines because the `print()` function is called inside the `for` loop. This means that the output of the `for` loop is printed before the `print()` statement that is outside of the `for` loop.

The reason why the list contains `None` values is because the `print()` function returns `None`. This is because the `print()` function does not return anything. It simply prints its arguments to the console.

To get the output that you want, you can use the `list()` function to convert the output of the `for` loop to a list. For example, you could use the following code:

```
names = ['ali', 'parham', 'hasan', 'farhad']
print('names: ', list(print(i) for i in names))
```

This code will print the following output:

```
names:  ['ali', 'parham', 'hasan', 'farhad']
```

## Evaulation
It's essential to evaluate your model to understand its performance. Evaluation can be done in an automated way using evaluation metrics like F1 or Rouge. You can also leverage human evaluation methods. Human evaluation methods involve asking humans to rate the quality of the LLM's answers. This can be done through crowdsourcing or by having experts evaluate the responses. Some standard human evaluation metrics include fluency, coherence, relevance, and informativeness. Often you want to choose a mix of evaluation metrics to get a good understanding of your model performance. Below you will find an example of how you can do the evaluation.

In this example you will be using [sequence-evaluate](https://pypi.org/project/sequence-evaluate/) to evaluation the tuned model.

In [82]:
from seq_eval import SeqEval
evaluator = SeqEval()

Earlier in the notebook, you created a train and eval dataset. Now it's time to take some of the eval data. You will use the questions to get a response from our tuned model, and the answers we will use as a reference:

- **Candidates**: Answers generated by the tuned model.
- **References**: Original answers that we will use to compare.

In [83]:
eval_set = evaluation.sample(n=10) # you can change the number of rows you want to use
evaluation_question = eval_set["input_text"]
evaluation_answer = eval_set["output_text"]

Now you can go ahead and generate candidates using the tuned model based on the questions you took from the eval dataset.

In [84]:
from tqdm import tqdm
import copy

In [85]:
candidates_deployed = []
candidates_original = []

for i in tqdm(evaluation_question):
    response_deployed = deployed_model.predict(i, max_output_tokens=1024, temperature=0.3)
    response_original = model.predict(i, max_output_tokens=1024, temperature=0.3)
    candidates_deployed.append(response_deployed.text)
    candidates_original.append(response_original.text)
    
len(candidates_deployed), len(candidates_original)

100%|██████████| 10/10 [01:16<00:00,  7.65s/it]


(10, 10)

You will also have to create a list of our references. These will you use to evaluate the model's performance.

In [86]:
references = evaluation_answer.tolist()

len(references)

10

Next you will generate the evaluation metrics. `evaluator.evaluate` will return a few eval metrics. Some of the important ones are:
- [Blue](https://en.wikipedia.org/wiki/BLEU): The BLEU evaluation metric is a measure of the similarity between a machine-generated text and a human-written reference text.
- [Rouge](https://en.wikipedia.org/wiki/ROUGE_(metric)): The ROUGE evaluation metric is a measure of the overlap between a machine-generated text and a human-written reference text.

In [87]:
scores_deployed = copy.deepcopy(evaluator.evaluate(candidates_deployed, references, verbose=False))
scores_original = copy.deepcopy(evaluator.evaluate(candidates_original, references, verbose=False))

In [88]:
pd.DataFrame.from_dict([scores_deployed, scores_original])

,bleu_1,bleu_2,bleu_3,bleu_4,rouge_1_precision,rouge_1_recall,rouge_1_f1,rouge_2_precision,rouge_2_recall,rouge_2_f1,rouge_l_precision,rouge_l_recall,rouge_l_f1,inter_dist1,inter_dist2,intra_dist1,intra_dist2,semantic_textual_similarity
0,0.109352,0.074284,0.056022,0.039543,0.417484,0.180539,0.228100,0.180354,0.081598,0.099006,0.404529,0.174862,0.220679,0.008866,0.080395,0.097889,0.339881,0.617321
1,0.103987,0.043883,0.025209,0.015407,0.217020,0.155215,0.155871,0.040225,0.030331,0.030176,0.209006,0.146231,0.148063,0.008210,0.079900,0.074809,0.314691,0.561274


Inne metody ewaluacji:
* human
* openai eval